In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()

In [2]:
def SimpleSparseTensorFrom(x):
  """Create a very simple SparseTensor with dimensions (batch, time).
  Args:
    x: a list of lists of type int
  Returns:
    x_ix and x_val, the indices and values of the SparseTensor<2>.
  """
  x_ix = []
  x_val = []
  for batch_i, batch in enumerate(x):
    for time, val in enumerate(batch):
      x_ix.append([batch_i, time])
      x_val.append(val)
  x_shape = [len(x), np.asarray(x_ix).max(0)[1]+1]
  x_ix = tf.constant(x_ix, tf.int64)
  x_val = tf.constant(x_val, tf.int32)
  x_shape = tf.constant(x_shape, tf.int64)

  return tf.SparseTensor(x_ix, x_val, x_shape)

In [16]:
print(SimpleSparseTensorFrom([[0,1,2,3,4,5,6], [9,6,7]]))

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [1 0]
 [1 1]
 [1 2]], shape=(10, 2), dtype=int64), values=tf.Tensor([0 1 2 3 4 5 6 9 6 7], shape=(10,), dtype=int32), dense_shape=tf.Tensor([2 7], shape=(2,), dtype=int64))


In [3]:
class CTCLossTest(tf.test.TestCase):

  def _testCTCLoss(self, inputs, seq_lens, labels,
                   loss_truth, expected_err_re=None):

    inputs_t = tf.constant(inputs)

    with self.test_session(use_gpu=False) as sess:
      loss = tf.nn.ctc_loss(inputs=inputs_t,
                            labels=labels,
                            sequence_length=seq_lens)

      self.assertShapeEqual(loss_truth, loss)
      print("Expected loss: ",loss_truth)
      print("Loss computed by ctc function: ", loss)

  def testBasic(self):
    """Test two batch entries."""
    # max_time_steps == 7
    depth = 6

    # seq_len_0 == 5
    targets_0 = [0, 1, 2, 1, 0]
    loss_log_prob_0 = 0
    # dimensions are time x depth
    input_prob_matrix_0 = np.asarray(
        [[1, 0., 0., 0., 0., 0.],
         [0., 1, 0., 0., 0., 0.],
         [0., 0., 1, 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1, 0., 0., 0., 0., 0.]],
        dtype=np.float32)
    input_log_prob_matrix_0 = np.log(input_prob_matrix_0)

    # seq_len_1 == 5
    targets_1 = [0, 1, 2, 1, 3]
    loss_log_prob_1 = 0
    # dimensions are time x depth

    input_prob_matrix_1 = np.asarray(
        [[1, 0., 0., 0., 0., 0.],
         [0., 1, 0., 0., 0., 0.],
         [0., 0., 1, 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0, 0.2, 0.3, 0.5, 0., 0.]],
        dtype=np.float32)
    input_log_prob_matrix_1 = np.log(input_prob_matrix_1)

    # len max_time_steps array of 2 x depth matrices
    inputs = [np.vstack([input_log_prob_matrix_0[t, :],
                         input_log_prob_matrix_1[t, :]])
              for t in range(5)] + 2 * [np.nan*np.ones((2, depth), np.float32)]

    # convert inputs into [max_time x batch_size x depth tensor] Tensor
    inputs = np.asarray(inputs, dtype=np.float32)

    # len batch_size array of label vectors
    labels = SimpleSparseTensorFrom([targets_0, targets_1])

    # batch_size length vector of sequence_lengths
    seq_lens = np.array([5, 5], dtype=np.int32)

    # output: batch_size length vector of negative log probabilities
    loss_truth = np.array([-loss_log_prob_0, -loss_log_prob_1], np.float32)


    self._testCTCLoss(inputs, seq_lens, labels, loss_truth)

In [4]:
ctc_loss = CTCLossTest()

In [5]:
ctc_loss.testBasic()

/home/neo/MS/TA/TCS_TL/env/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log
/home/neo/MS/TA/TCS_TL/env/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in log


Expected loss:  [0. 0.]
Loss computed by ctc function:  tf.Tensor([-0.         0.6931472], shape=(2,), dtype=float32)
